In [1]:
import time
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('dark_background')

import h5py
import numpy as np
import tenseal as ts
import torch
import torch.nn as nn
import torch.nn.functional as F
from icecream import ic
ic.configureOutput(includeContext=True)
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, Dataset

print(f'torch version: {torch.__version__}')
print(f'tenseal version: {ts.__version__}')

project_path = Path.cwd().parent
print(f'project_path: {project_path}')

torch version: 1.8.1+cu102
tenseal version: 0.3.5
project_path: /home/dk/Desktop/split-learning-1D-HE


## Dataset

In [2]:
class ECG(Dataset):
    """The class used by the client to load the dataset

    Args:
        Dataset ([type]): [description]
    """
    def __init__(self, train=True):
        if train:
            with h5py.File(project_path/'data/train_ecg.hdf5', 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(project_path/'data/test_ecg.hdf5', 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

In [3]:
batch_size = 4
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Training loop

In [4]:
epoch = 10
lr = 0.001
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f'device: {torch.cuda.get_device_name(0)}')

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def train(model, save_weight_path: str):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    train_losses = list()
    train_accs = list()
    train_times = list()

    test_losses = list()
    test_accs = list()
    test_times = list()

    best_test_acc = 0  # best test accuracy 

    for e in range(epoch):
        train_start = time.time()
        print("Epoch {} - ".format(e+1), end='')

        # train
        train_loss = 0.0
        correct, total = 0, 0
        for i, batch in enumerate(train_loader):
            x, label = batch  # get feature and label from a batch
            x, label = x.to(device), label.to(device)  # send to device
            optimizer.zero_grad()  # init all grads to zero
            output = model(x)  # forward propagation
            loss = criterion(output, label)  # calculate loss
            loss.backward()  # backward propagation
            optimizer.step()  # weight update

            train_loss += loss.item()
            correct += torch.sum(output.argmax(dim=1) == label).item()
            total += len(label)

        train_losses.append(train_loss / len(train_loader))
        train_accs.append(correct / total)
        train_end = time.time()
        train_times.append(train_end-train_start)
        print("loss: {:.4f}, acc: {:.2f}%".format(train_losses[-1], train_accs[-1]*100), end=' / ')
    
        # test
        test_start = time.time()
        with torch.no_grad():
            test_loss = 0.0
            correct, total = 0, 0
            for _, batch in enumerate(test_loader):
                x, label = batch
                x, label = x.to(device), label.to(device)
                output = model(x)
                loss = criterion(output, label)
                
                test_loss += loss.item()
                correct += torch.sum(output.argmax(dim=1) == label).item()
                total += len(label)
            test_losses.append(test_loss / len(test_loader))
            test_acc = correct / total
            test_accs.append(test_acc)
        test_end = time.time()
        test_times.append(test_end-test_start)
        print("test_loss: {:.4f}, test_acc: {:.2f}%".format(test_losses[-1], test_accs[-1]*100))
        if test_accs[-1] > best_test_acc:
            best_test_acc = test_acc
            torch.save(ecgnet.state_dict(), save_weight_path)  # save trained weights
            print(f"found new best test accuracy: {test_acc*100:.2f}%, save model")

    return train_losses, train_accs, train_times, test_losses, test_accs, test_times

device: NVIDIA GeForce GTX 1070 Ti


## Models

### Model 1: 512 (activation maps length)

In [5]:
class ECGConv1D512(nn.Module):
    def __init__(self):
        super(ECGConv1D512, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 16 = 512

        self.linear1 = nn.Linear(512, 128)
        self.relu3 = nn.LeakyReLU()
        self.linear2 = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 512)
        x = self.linear1(x)
        x = self.relu3(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

ecgnet = ECGConv1D512()
train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(ecgnet.to(device),
                                                'weights/trained_weight_512_2.pth')

Epoch 1 - loss: 1.1628, acc: 74.79% / test_loss: 1.0431, test_acc: 86.99%
found new best test accuracy: 86.99%, save model
Epoch 2 - loss: 1.0341, acc: 87.33% / test_loss: 0.9923, test_acc: 91.57%
found new best test accuracy: 91.57%, save model
Epoch 3 - loss: 0.9953, acc: 91.06% / test_loss: 0.9877, test_acc: 91.84%
found new best test accuracy: 91.84%, save model
Epoch 4 - loss: 0.9921, acc: 91.30% / test_loss: 0.9706, test_acc: 93.47%
found new best test accuracy: 93.47%, save model
Epoch 5 - loss: 0.9823, acc: 92.34% / test_loss: 0.9688, test_acc: 93.67%
found new best test accuracy: 93.67%, save model
Epoch 6 - loss: 0.9781, acc: 92.71% / test_loss: 0.9681, test_acc: 93.70%
found new best test accuracy: 93.70%, save model
Epoch 7 - loss: 0.9742, acc: 93.07% / test_loss: 0.9668, test_acc: 93.91%
found new best test accuracy: 93.91%, save model
Epoch 8 - loss: 0.9740, acc: 93.18% / test_loss: 0.9658, test_acc: 93.89%
Epoch 9 - loss: 0.9709, acc: 93.37% / test_loss: 0.9639, test_acc

### Model 2: 256

In [6]:
class ECGConv1D256(nn.Module):
    def __init__(self):
        super(ECGConv1D256, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 8, 5, padding=2)  # 64 x 8
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 8 = 256

        self.linear1 = nn.Linear(256, 128)
        self.relu3 = nn.LeakyReLU()
        self.linear2 = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 256)
        x = self.linear1(x)
        x = self.relu3(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

ecgnet = ECGConv1D256()
train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(ecgnet.to(device), 
                            'weights/trained_weight_256_2.pth')

Epoch 1 - loss: 1.2078, acc: 70.23% / test_loss: 1.0692, test_acc: 84.57%
found new best test accuracy: 84.57%, save model
Epoch 2 - loss: 1.0606, acc: 84.72% / test_loss: 1.0276, test_acc: 88.05%
found new best test accuracy: 88.05%, save model
Epoch 3 - loss: 1.0407, acc: 86.52% / test_loss: 1.0249, test_acc: 88.09%
found new best test accuracy: 88.09%, save model
Epoch 4 - loss: 1.0349, acc: 86.98% / test_loss: 1.0215, test_acc: 88.34%
found new best test accuracy: 88.34%, save model
Epoch 5 - loss: 1.0279, acc: 87.58% / test_loss: 1.0143, test_acc: 88.95%
found new best test accuracy: 88.95%, save model
Epoch 6 - loss: 1.0255, acc: 87.93% / test_loss: 1.0089, test_acc: 89.45%
found new best test accuracy: 89.45%, save model
Epoch 7 - loss: 1.0225, acc: 88.15% / test_loss: 1.0098, test_acc: 89.39%
Epoch 8 - loss: 1.0226, acc: 88.12% / test_loss: 1.0093, test_acc: 89.35%
Epoch 9 - loss: 1.0226, acc: 88.14% / test_loss: 1.0085, test_acc: 89.42%
Epoch 10 - loss: 1.0196, acc: 88.44% / t

### Model 3: 128

In [7]:
class ECGConv1D128(nn.Module):
    def __init__(self):
        super(ECGConv1D128, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 4, 5, padding=2)  # 64 x 4
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 4 = 128

        self.linear1 = nn.Linear(128, 128)
        self.relu3 = nn.LeakyReLU()
        self.linear2 = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 128)
        x = self.linear1(x)
        x = self.relu3(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

ecgnet = ECGConv1D128()
train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(ecgnet.to(device), 
                            'weights/trained_weight_128_2.pth')

Epoch 1 - loss: 1.2370, acc: 67.47% / test_loss: 1.0895, test_acc: 82.45%
found new best test accuracy: 82.45%, save model
Epoch 2 - loss: 1.0658, acc: 84.18% / test_loss: 1.0363, test_acc: 86.91%
found new best test accuracy: 86.91%, save model
Epoch 3 - loss: 1.0456, acc: 86.04% / test_loss: 1.0274, test_acc: 87.83%
found new best test accuracy: 87.83%, save model
Epoch 4 - loss: 1.0349, acc: 87.01% / test_loss: 1.0239, test_acc: 88.05%
found new best test accuracy: 88.05%, save model
Epoch 5 - loss: 1.0311, acc: 87.33% / test_loss: 1.0134, test_acc: 89.02%
found new best test accuracy: 89.02%, save model
Epoch 6 - loss: 1.0270, acc: 87.72% / test_loss: 1.0119, test_acc: 89.22%
found new best test accuracy: 89.22%, save model
Epoch 7 - loss: 1.0239, acc: 87.97% / test_loss: 1.0130, test_acc: 89.11%
Epoch 8 - loss: 1.0238, acc: 87.95% / test_loss: 1.0110, test_acc: 89.27%
found new best test accuracy: 89.27%, save model
Epoch 9 - loss: 1.0201, acc: 88.35% / test_loss: 1.0099, test_acc

### Model 4: 64

In [8]:
class ECGConv1D64(nn.Module):
    def __init__(self):
        super(ECGConv1D64, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 2, 5, padding=2)  # 64 x 2
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 2 = 64

        self.linear1 = nn.Linear(64, 128)
        self.relu3 = nn.LeakyReLU()
        self.linear2 = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 64)
        x = self.linear1(x)
        x = self.relu3(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

ecgnet = ECGConv1D64()
train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(ecgnet.to(device),
                                'weights/trained_weight_64_2.pth')

Epoch 1 - loss: 1.2298, acc: 67.66% / test_loss: 1.0946, test_acc: 81.53%
found new best test accuracy: 81.53%, save model
Epoch 2 - loss: 1.1001, acc: 80.45% / test_loss: 1.0754, test_acc: 82.83%
found new best test accuracy: 82.83%, save model
Epoch 3 - loss: 1.0759, acc: 82.96% / test_loss: 1.0295, test_acc: 87.75%
found new best test accuracy: 87.75%, save model
Epoch 4 - loss: 1.0419, acc: 86.42% / test_loss: 1.0185, test_acc: 88.68%
found new best test accuracy: 88.68%, save model
Epoch 5 - loss: 1.0331, acc: 87.26% / test_loss: 1.0162, test_acc: 88.85%
found new best test accuracy: 88.85%, save model
Epoch 6 - loss: 1.0287, acc: 87.61% / test_loss: 1.0158, test_acc: 88.95%
found new best test accuracy: 88.95%, save model
Epoch 7 - loss: 1.0252, acc: 87.97% / test_loss: 1.0121, test_acc: 89.14%
found new best test accuracy: 89.14%, save model
Epoch 8 - loss: 1.0230, acc: 88.04% / test_loss: 1.0148, test_acc: 88.86%
Epoch 9 - loss: 1.0213, acc: 88.21% / test_loss: 1.0143, test_acc